In [54]:
import numpy as np # for Linear algebra
import pandas as pd # for data manipulation/CSV I/O
import matplotlib.pyplot as plt # for plotting Graphs
import seaborn as sns # for interactive graphs
pd.options.mode.chained_assignment = None    # To avoid the SettingWithCopyWarning

In [55]:
deliveries=pd.read_csv("dataset/deliveries.csv")
matches =pd.read_csv("dataset/matches.csv")

In [56]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [57]:
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [58]:
deliveries.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [59]:
column_heads = ['match_id','batsman','batsman_runs']
batsman = deliveries[column_heads]
bat = deliveries[column_heads]
batsman.sample()

,match_id,batsman,batsman_runs
117125,494,F du Plessis,2


In [60]:
batsman["Fours"] = np.where(batsman["batsman_runs"] == 4, 1, 0)
batsman["Sixes"] = np.where(batsman["batsman_runs"] == 6, 1, 0)
print(batsman.head())

   match_id    batsman  batsman_runs  Fours  Sixes
0         1  DA Warner             0      0      0
1         1  DA Warner             0      0      0
2         1  DA Warner             4      1      0
3         1  DA Warner             0      0      0
4         1  DA Warner             0      0      0


In [61]:
batsmen_info = batsman.groupby(['batsman']).agg(
    {'match_id': lambda x: x.nunique(), 'batsman_runs': 'sum', 'Fours': 'sum', 'Sixes': 'sum'}).reset_index()

print(batsmen_info.sample())

         batsman  match_id  batsman_runs  Fours  Sixes
26  AC Gilchrist        80          2069    239     92


In [62]:
temp = deliveries.groupby(['match_id','batsman'])
hs = temp.sum().reset_index().groupby('batsman').agg({'batsman_runs': 'max'}).reset_index()
thirty = temp.sum().reset_index().groupby(['match_id', 'batsman']).agg(
    {'batsman_runs': lambda x: 1 if (x > 29).any() else 0}).reset_index().groupby('batsman').agg(
    {'batsman_runs': 'sum'}).reset_index()

balls = temp.count().reset_index().groupby('batsman').agg({'batsman_runs': 'sum'}).reset_index()

thirty = thirty.rename(columns={'batsman_runs': "30s"})
hs = hs.rename(columns={'batsman_runs': "HS"})
balls = balls.rename(columns={'batsman_runs': "Ball faced"})

batsmen_info = pd.merge(batsmen_info, hs, on=['batsman'])
batsmen_info = pd.merge(batsmen_info, thirty, on=['batsman'])
batsmen_info = pd.merge(batsmen_info, balls, on=['batsman'])

print(batsmen_info.sample())

        batsman  match_id  batsman_runs  Fours  Sixes  HS  30s  Ball faced
72  BCJ Cutting        14           222     14     18  39    2         129


In [63]:
batsmen_info['SR'] = np.around((batsmen_info['batsman_runs'] / batsmen_info['Ball faced']) * 100, 2)
print(batsmen_info.head())

          batsman  match_id  batsman_runs  Fours  Sixes  HS  30s  Ball faced  \
0  A Ashish Reddy        23           280     16     15  36    2         196   
1      A Chandila         2             4      0      0   4    0           7   
2        A Chopra         6            53      7      0  24    0          75   
3     A Choudhary         3            25      1      1  15    0          20   
4     A Dananjaya         1             4      0      0   4    0           5   

       SR  
0  142.86  
1   57.14  
2   70.67  
3  125.00  
4   80.00  


In [64]:
batsmen_info = batsmen_info.rename(columns={'batsman': 'Batsman', 'batsman_runs': 'Runs', 'match_id': 'Matches'})
sequence = ['Batsman', 'Matches', 'Runs', 'SR', '30s', 'HS', 'Sixes', 'Fours']
batsmen_info = batsmen_info[sequence]
print(batsmen_info.head())

          Batsman  Matches  Runs      SR  30s  HS  Sixes  Fours
0  A Ashish Reddy       23   280  142.86    2  36     15     16
1      A Chandila        2     4   57.14    0   4      0      0
2        A Chopra        6    53   70.67    0  24      0      7
3     A Choudhary        3    25  125.00    0  15      1      1
4     A Dananjaya        1     4   80.00    0   4      0      0
